In [1]:
import boto3
import pandas as pd

bucket = "ai-bmi-predictor"
key = "test-data/eff_testingB.csv"

s3 = boto3.client("s3")
obj = s3.get_object(Bucket=bucket, Key=key)

data = pd.read_csv(obj["Body"])

data.head()


,photo_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,hip,leg-length,shoulder-breadth,shoulder-to-crotch,thigh,waist,wrist,gender,height_cm,weight_kg
0,5e09e1b0d43b6c430709a513f594c591,0.109534,-0.016982,-0.146092,0.516881,0.189749,-0.067232,-0.048589,-0.022137,0.045580,...,91.273094,77.553963,36.633358,64.253914,47.014259,77.113548,16.389431,male,170.00,61.8
1,7e6a57e2fcabf518e9007a77d4cc4960,0.094942,0.000988,-0.126504,0.834527,0.315265,0.002205,-0.067837,-0.093690,-0.021319,...,91.707520,76.172325,34.205143,61.726360,48.404037,73.617821,14.480244,female,169.00,54.5
2,4ee8251ae7e4ad42c75644db390fc5c5,-0.032353,-0.051524,-0.129219,0.430257,0.209280,-0.131291,0.039772,-0.062221,0.072620,...,89.962387,74.400047,31.023907,56.728245,47.468864,72.155304,15.670779,female,164.59,52.1
3,6367d327f96b951e4a498c88d615e8a4,0.056546,0.007736,-0.147426,0.731932,0.264191,-0.050348,-0.145324,-0.088448,-0.021515,...,96.964554,76.552322,36.380489,63.264416,47.635548,82.428825,16.714935,male,168.00,67.4
4,8b16daea6768b92fb27997f76215aa5f,0.004628,-0.047994,-0.155691,0.587151,0.235884,-0.101305,-0.065924,-0.134367,0.021547,...,115.346634,85.163284,39.959301,72.723900,61.206940,115.251938,19.627054,male,184.30,111.3


In [2]:
# =========================
# Volume feature engineering
# =========================

import boto3  # AWS SDK for Python (S3 access)
import numpy as np  # fast array/mask operations
import pandas as pd  # dataframe ops
from PIL import Image  # read PNG images
from io import BytesIO  # convert S3 bytes -> file-like object
from botocore.exceptions import ClientError  # catch S3 missing-key errors

# -------------------------
# CONFIG (edit if needed)
# -------------------------

MASK_BUCKET = "amazon-bodym"            # S3 bucket that contains mask images
FRONT_MASK_PREFIX = "testB/mask/"      # front masks folder (from your 1st screenshot)
SIDE_MASK_PREFIX = "testB/mask_left/"  # side masks folder (from your 2nd screenshot)
PHOTO_ID_COL = "photo_id"              # column in your CSV that maps to mask filenames
OUTPUT_COL = "volume"                  # name for the new feature column

# -------------------------
# S3 client (reused)
# -------------------------

print("Creating S3 client...")  # track progress
s3 = boto3.client("s3")  # create an S3 client

# -----------------------------------------
# Helper: standardize photo_id -> filename
# -----------------------------------------

print("Inferring photo_id padding length...")  # track progress
photo_id_series = data[PHOTO_ID_COL].astype(str).fillna("")  # ensure string + no NaN
photo_id_series = photo_id_series.str.strip()  # remove whitespace
photo_id_series = photo_id_series.str.replace(".png", "", regex=False)  # drop extension if present
photo_id_series = photo_id_series.str.split(".", n=1).str[0]  # drop trailing ".0" etc (common if numeric CSV)
pad_len = int(photo_id_series.str.len().max()) if len(photo_id_series) else 0  # infer max length
pad_len = max(pad_len, 4)  # default to 4 (matches your example like "0021")
print(f"Using zero-pad length = {pad_len}")  # show chosen padding

def normalize_photo_id(photo_id, pad_length):  # define normalizer
    pid = str(photo_id).strip()  # convert to string and trim spaces
    pid = pid.replace(".png", "")  # remove .png if included
    pid = pid.split(".", 1)[0]  # remove any trailing decimals (e.g., "21.0" -> "21")
    if pid.isdigit():  # only pad if it's purely digits
        pid = pid.zfill(pad_length)  # left-pad with zeros
    return pid  # return normalized id like "0021"

# -----------------------------------------
# Helper: read mask PNG from S3 into boolean
# -----------------------------------------

def load_mask_bool_from_s3(bucket, key):  # function to load a mask file
    print(f"    Downloading: s3://{bucket}/{key}")  # track downloads
    obj = s3.get_object(Bucket=bucket, Key=key)  # fetch object bytes from S3
    img = Image.open(BytesIO(obj["Body"].read()))  # open image from bytes
    img = img.convert("L")  # convert to grayscale
    arr = np.array(img)  # convert to numpy array
    mask_bool = arr > 0  # treat any non-zero pixel as "body"
    return mask_bool  # return boolean mask

# -----------------------------------------
# Helper: compute mean width across rows
# -----------------------------------------

def mean_row_width(mask_bool):  # compute average body width across horizontal rows
    if mask_bool.size == 0:  # guard: empty image
        return np.nan  # cannot compute
    row_has_body = mask_bool.any(axis=1)  # rows where body exists
    if not row_has_body.any():  # guard: no body pixels
        return np.nan  # cannot compute
    first = mask_bool.argmax(axis=1)  # first True index per row (0 if none)
    last = mask_bool.shape[1] - 1 - np.flip(mask_bool, axis=1).argmax(axis=1)  # last True index per row
    widths = (last - first + 1).astype(float)  # width per row (in pixels)
    widths[~row_has_body] = np.nan  # ignore rows with no body
    return float(np.nanmean(widths))  # average width across valid rows

# -----------------------------------------
# Helper: compute height from front mask
# -----------------------------------------

def body_height(mask_bool):  # compute vertical extent (top-to-bottom)
    if mask_bool.size == 0:  # guard: empty image
        return np.nan  # cannot compute
    ys, xs = np.where(mask_bool)  # get coordinates of body pixels
    if ys.size == 0:  # guard: no body pixels
        return np.nan  # cannot compute
    h = (ys.max() - ys.min() + 1).astype(float)  # height in pixels
    return float(h)  # return height

# -----------------------------------------
# Per-record feature computation
# -----------------------------------------

def compute_volume_for_photo_id(photo_id):  # compute volume + debug parts for one record
    pid = normalize_photo_id(photo_id, pad_len)  # normalize id to match PNG filename
    front_key = f"{FRONT_MASK_PREFIX}{pid}.png"  # build front mask key
    side_key = f"{SIDE_MASK_PREFIX}{pid}.png"  # build side mask key

    try:
        front_mask = load_mask_bool_from_s3(MASK_BUCKET, front_key)  # load front mask
    except ClientError as e:  # handle missing front file
        print(f"    ERROR: missing front mask for {pid}: {e}")  # print error
        return np.nan, np.nan, np.nan, np.nan  # return NaNs

    try:
        side_mask = load_mask_bool_from_s3(MASK_BUCKET, side_key)  # load side mask
    except ClientError as e:  # handle missing side file
        print(f"    ERROR: missing side mask for {pid}: {e}")  # print error
        return np.nan, np.nan, np.nan, np.nan  # return NaNs

    front_w = mean_row_width(front_mask)  # front width (body looks wide from front)
    side_w = mean_row_width(side_mask)  # side width (body looks thick from side)
    avg_w = (front_w + side_w) / 2.0  # average width as instructed
    h = body_height(front_mask)  # height from front mask (vertical extent)

    raw_volume = avg_w * h  # volume ≈ average width × height
    norm_volume = raw_volume / (h ** 2) if (h is not None and not np.isnan(h) and h > 0) else np.nan  # normalize

    return norm_volume, front_w, side_w, h  # return volume + components for debugging

# -----------------------------------------
# Run across the dataframe and add column(s)
# -----------------------------------------

print("Starting volume computation for all records...")  # track progress
volumes = []  # store normalized volumes
front_ws = []  # store front widths (debug)
side_ws = []  # store side widths (debug)
heights = []  # store heights (debug)

n = len(data)  # total rows
print(f"Total rows to process: {n}")  # show total

for i, photo_id in enumerate(data[PHOTO_ID_COL].tolist(), start=1):  # iterate through photo_ids
    print(f"\nRecord {i}/{n} | photo_id={photo_id}")  # show per-record progress
    v, fw, sw, h = compute_volume_for_photo_id(photo_id)  # compute features
    print(f"    front_width={fw} | side_width={sw} | height={h} | volume={v}")  # print computed values
    volumes.append(v)  # append volume
    front_ws.append(fw)  # append front width
    side_ws.append(sw)  # append side width
    heights.append(h)  # append height

    if i % 50 == 0:  # every 50 rows
        print(f"\nProcessed {i}/{n} records so far...")  # periodic progress update

print("\nAttaching new feature columns to dataframe...")  # track progress
data[OUTPUT_COL] = volumes  # add normalized volume feature
data["front_width"] = front_ws  # optional debug column
data["side_width"] = side_ws  # optional debug column
data["mask_height"] = heights  # optional debug column

print("Done.")  # final status
print("Preview of updated dataframe:")  # show preview message
print(data[[PHOTO_ID_COL, OUTPUT_COL, "front_width", "side_width", "mask_height"]].head())  # preview new columns


Creating S3 client...
Inferring photo_id padding length...
Using zero-pad length = 32
Starting volume computation for all records...
Total rows to process: 1160

Record 1/1160 | photo_id=5e09e1b0d43b6c430709a513f594c591
    Downloading: s3://amazon-bodym/testB/mask/5e09e1b0d43b6c430709a513f594c591.png
    Downloading: s3://amazon-bodym/testB/mask_left/5e09e1b0d43b6c430709a513f594c591.png
    front_width=254.09480812641084 | side_width=94.09688581314879 | height=886.0 | volume=0.1964964412751465

Record 2/1160 | photo_id=7e6a57e2fcabf518e9007a77d4cc4960
    Downloading: s3://amazon-bodym/testB/mask/7e6a57e2fcabf518e9007a77d4cc4960.png
    Downloading: s3://amazon-bodym/testB/mask_left/7e6a57e2fcabf518e9007a77d4cc4960.png
    front_width=219.09666666666666 | side_width=93.36610169491526 | height=900.0 | volume=0.17359042686754553

Record 3/1160 | photo_id=4ee8251ae7e4ad42c75644db390fc5c5
    Downloading: s3://amazon-bodym/testB/mask/4ee8251ae7e4ad42c75644db390fc5c5.png
    Downloading: s

In [3]:
# Drop the intermediate/debug columns (ignore if they don't exist)
cols_to_drop = ["front_width", "side_width", "mask_height"]  # columns to remove
print("Dropping columns:", cols_to_drop)  # track process
data.drop(columns=cols_to_drop, inplace=True, errors="ignore")  # drop safely

Dropping columns: ['front_width', 'side_width', 'mask_height']


In [4]:
data.head()

,photo_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,leg-length,shoulder-breadth,shoulder-to-crotch,thigh,waist,wrist,gender,height_cm,weight_kg,volume
0,5e09e1b0d43b6c430709a513f594c591,0.109534,-0.016982,-0.146092,0.516881,0.189749,-0.067232,-0.048589,-0.022137,0.045580,...,77.553963,36.633358,64.253914,47.014259,77.113548,16.389431,male,170.00,61.8,0.196496
1,7e6a57e2fcabf518e9007a77d4cc4960,0.094942,0.000988,-0.126504,0.834527,0.315265,0.002205,-0.067837,-0.093690,-0.021319,...,76.172325,34.205143,61.726360,48.404037,73.617821,14.480244,female,169.00,54.5,0.173590
2,4ee8251ae7e4ad42c75644db390fc5c5,-0.032353,-0.051524,-0.129219,0.430257,0.209280,-0.131291,0.039772,-0.062221,0.072620,...,74.400047,31.023907,56.728245,47.468864,72.155304,15.670779,female,164.59,52.1,0.197037
3,6367d327f96b951e4a498c88d615e8a4,0.056546,0.007736,-0.147426,0.731932,0.264191,-0.050348,-0.145324,-0.088448,-0.021515,...,76.552322,36.380489,63.264416,47.635548,82.428825,16.714935,male,168.00,67.4,0.180588
4,8b16daea6768b92fb27997f76215aa5f,0.004628,-0.047994,-0.155691,0.587151,0.235884,-0.101305,-0.065924,-0.134367,0.021547,...,85.163284,39.959301,72.723900,61.206940,115.251938,19.627054,male,184.30,111.3,0.199326


In [ ]:
import boto3  # S3 client
from io import StringIO  # in-memory text buffer

print("Preparing to upload updated dataset to S3...")  # track process

s3_out_path = "s3://ai-bmi-predictor/test-data/eff_testingB_v2.csv"  # target S3 path
print("Target:", s3_out_path)  # show target

# ---- parse s3://bucket/key ----
out_bucket = s3_out_path.replace("s3://", "").split("/", 1)[0]  # extract bucket
out_key = s3_out_path.replace("s3://", "").split("/", 1)[1]     # extract key
print(f"Parsed -> bucket={out_bucket}, key={out_key}")  # confirm parsing

# ---- write CSV to memory ----
csv_buffer = StringIO()  # create in-memory buffer
print("Serializing dataframe to CSV (in-memory)...")  # track process
data.to_csv(csv_buffer, index=False)  # write dataframe as CSV text
csv_body = csv_buffer.getvalue()  # get CSV string content
print(f"CSV size (chars): {len(csv_body):,}")  # print rough size

# ---- upload to S3 ----
s3 = boto3.client("s3")  # create S3 client
print("Uploading to S3...")  # track process
s3.put_object(
    Bucket=out_bucket,                 # destination bucket
    Key=out_key,                       # destination key
    Body=csv_body.encode("utf-8"),     # file bytes
    ContentType="text/csv"             # content type
)  # upload

print("Upload complete ✅")  # done
print(f"Saved to: {s3_out_path}")  # confirm final path


Preparing to upload updated dataset to S3...
Target: s3://ai-bmi-predictor/test-data/eff_testingB_v2.csv
Parsed -> bucket=ai-bmi-predictor, key=test-data/eff_testingB_v2.csv
Serializing dataframe to CSV (in-memory)...
